In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from utils import u_arrays as ua
import cartopy 
import cartopy.crs as ccrs
import salem
%pylab inline
import pyproj

Populating the interactive namespace from numpy and matplotlib


In [2]:
files = ua.locate(".nc", '/users/global/cornkle/TRMMfiles')
# make a salem grid
proj = pyproj.Proj('+proj=merc +lat_0=0. +lon_0=0.')
# Transform lon, lats to the mercator projection
x, y = pyproj.transform(salem.wgs84, proj,[-18.5, 32], [3,21.5])
# take the min and max
xmax, xmin = np.max(x), np.min(x)
ymax, ymin = np.max(y), np.min(y)
# Count the number of pixels
dx = 5000
nx, r = divmod(xmax - xmin, dx)
ny, r = divmod(ymax - ymin, dx)
# Here one could add + 1 to be sure that the last pixel is always included
grid = salem.Grid(nxny=(nx, ny), dxdy=(dx, dx), ll_corner=(xmin, ymin), proj=proj)

In [3]:
len(files)

2711

In [4]:
lon, lat = grid.ll_coordinates
xi, yi = grid.ij_coordinates

In [10]:
tlon = []
tlat = []
for f in files:
    ff = xr.open_dataset(f)
    if np.sum(ff['t']) == 0:
    # print('no t, continue')
        continue
    tlon.append(ff['lon'].values.flatten().tolist())
    tlat.append(ff['lat'].values.flatten().tolist())

In [11]:
tlon = [item for sublist in tlon for item in sublist]
tlat = [item for sublist in tlat for item in sublist]

In [13]:
trmm_arr = np.zeros_like(lon)

In [ ]:
for lo3, la3 in zip(tlon,tlat):
    xm, ym = grid.transform(lo3, la3, crs=salem.wgs84)
    trmm_arr[int(np.round(ym)), int(np.round(xm))]+=1

In [78]:
path = '/users/global/cornkle/C_paper/wavelet/figs/'
fig = plt.figure(figsize=(9,7), dpi=400)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

plt.imshow(po30_arr, extent=(lon.min(), lon.max(), lat.min(), lat.max()), clim=(0,2), transform=ccrs.PlateCarree(), cmap='hot_r')
#plt.contourf(lon, lat, po30_arr, transform=ccrs.PlateCarree(), cmap='viridis_r', levels=np.arange(0,3,0.5) )
ax.coastlines()
plt.colorbar()
plt.title('Pixel count precip>30')
ax.add_feature(cartopy.feature.BORDERS, linestyle='--')

In [ ]:
np.sum(trmm_arr)